<a href="https://colab.research.google.com/github/moodlep/MLC_A3C/blob/main/a3c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]

     |████████████████████████████████| 448 kB 5.2 MB/s 


In [2]:
import os
import Box2D
import pyglet
import imageio
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'


In [3]:
import gym
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp


In [4]:
# The env - quick test: 

env = gym.make("LunarLander-v2")

s = env.reset()

for _ in range(5): 
  a = env.action_space.sample()
  next_state, reward, done, info = env.step(a)
  print(next_state, reward, a)


[ 0.01579113  1.4081995   0.79524124 -0.04177593 -0.01859165 -0.18812022
  0.          0.        ] -0.038094736960334774 2
[ 0.02357159  1.4066625   0.7855884  -0.06842816 -0.02604947 -0.14916997
  0.          0.        ] 0.14451482824213827 1
[ 0.03129129  1.4053156   0.7798752  -0.06002046 -0.03386489 -0.15632293
  0.          0.        ] -0.3237968686503507 2
[ 0.03917313  1.4044763   0.7954258  -0.03748155 -0.04102248 -0.1431656
  0.          0.        ] -2.3643076220101475 2
[ 0.04715729  1.4042549   0.80530965 -0.01004783 -0.04785093 -0.13658157
  0.          0.        ] -1.8916400234749233 2


In [5]:
class SharedAdam(torch.optim.Adam):
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.99), eps=1e-8,
                 weight_decay=0):
        super(SharedAdam, self).__init__(params, lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        # State initialization
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = 0
                state['exp_avg'] = torch.zeros_like(p.data)
                state['exp_avg_sq'] = torch.zeros_like(p.data)

                # share in memory
                state['exp_avg'].share_memory_()
                state['exp_avg_sq'].share_memory_()

In [6]:
env.action_space

Discrete(4)

### Actor - policy NN and value NN 
### data collection -> batch
### train: calculate loss 

In [7]:

class Policy(nn.Module):
	def __init__(self, state_dim,action_dim,hidden=100):
		super(Policy, self).__init__()

		self.l1 = nn.Linear(state_dim, hidden)
		self.l2 = nn.Linear(hidden, hidden)
		self.l3 = nn.Linear(hidden,action_dim)

	def forward(self, state):
		q = F.leaky_relu(self.l1(state))
		q = F.leaky_relu(self.l2(q))
		return F.softmax(self.l3(q), dim = 1)
	
	def get_action(self,state):
		with torch.no_grad():
			pol = self.forward(state)
			dist = torch.distributions.Categorical(pol)
		return dist.sample() #returns a batch of values
	
	def log_prob(self, state, actions):
		  # Part of the loss term
			pol = self.forward(state)
			log_prob = torch.distributions.Categorical(pol).log_prob(actions)
			return log_prob
	
	def entropy(self, state):
			pol = self.forward(state)
			return torch.distributions.Categorical(pol).entropy()
    


In [8]:
# create batch of states 
batch_states = torch.rand(5, env.observation_space.shape[0])

policy = Policy(env.observation_space.shape[0], env.action_space.n)
policy(batch_states).data

tensor([[0.2250, 0.2561, 0.2576, 0.2612],
        [0.2141, 0.2636, 0.2563, 0.2661],
        [0.2091, 0.2661, 0.2586, 0.2662],
        [0.2164, 0.2614, 0.2632, 0.2590],
        [0.2224, 0.2532, 0.2632, 0.2612]])

In [9]:
batch_actions = policy.get_action(batch_states)
batch_actions

tensor([0, 3, 3, 1, 3])

In [10]:
policy.log_prob(batch_states, batch_actions)

tensor([-1.4916, -1.3240, -1.3234, -1.3418, -1.3425],
       grad_fn=<SqueezeBackward1>)

In [11]:
class Critic(nn.Module):
    def __init__(self, state_dim,hidden=100):
      super(Critic, self).__init__()
    
      self.l1 = nn.Linear(state_dim, hidden)
      self.l2 = nn.Linear(hidden, hidden)
      self.l3 = nn.Linear(hidden,1)

    def forward(self, state):
      q = F.leaky_relu(self.l1(state))
      q = F.leaky_relu(self.l2(q))
      return self.l3(q)



In [ ]:
class ActorCritic(mp.Process):
	def __init__(self,env_name,glb_critic,glb_policy,opt_crt,opt_pol,max_step=100):
		self.env = gym.make(env_name)
		self.t = 0
		self.max_step = max_step

		self.actor = Policy(self.env.observation_space.shape[0], self.env.action_space.n)
		self.critic = Critic(self.env.observation_space.shape[0])
		self.global_critic = global_critic
		self.global_policy = global_policy
	
	def run(self):

		# 1. Sync local from global
		self.actor.load_state_dict(self.global_policy.state_dict())
		self.critic.load_state_dict(self.global_critic.state_dict())
	
		# 2. Create a rollout


		# 3. 


    
    


In [ ]:
# worker process
# Input: A2C network, env, no of steps, 

# 1. 
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n


global_critic = Critic(state_dim)
global_policy = Policy(state_dim,action_dim)
global_critic.share_memory()
global_policy.share_memory()

global_opt_crt = SharedAdam(global_critic.parameters())
global_opt_pol = SharedAdam(global_policy.parameters())


global_ctr = mp.Value('i',0)
lock = mp.Lock()

pr = [mp.Process(target=test,args=(a,)) for _ in range(5)]

for p in pr:
    p.start()

    
for p in pr:
    p.join()


Policy(
  (l1): Linear(in_features=8, out_features=100, bias=True)
  (l2): Linear(in_features=100, out_features=100, bias=True)
  (l3): Linear(in_features=100, out_features=4, bias=True)
)